In [ ]:
!pip install split-folders
import splitfolders
from google.colab import drive
drive.mount('/content/drive')
splitfolders.ratio("/content/drive/MyDrive/PhDProject/colon_image_sets",
            output="splitted_data",
            seed=42,
            ratio=(.7, .2, .1),
            group_prefix=None,
            move=False)

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, GlobalAveragePooling2D, Input, Concatenate, Add
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

def fire_module(x, squeeze_filters, expand1x1_filters, expand3x3_filters, dilation_rate=1):
    squeeze = Conv2D(filters=squeeze_filters, kernel_size=(1, 1), activation='relu')(x)

    expand1x1 = Conv2D(filters=expand1x1_filters, kernel_size=(1, 1), activation='relu')(squeeze)
    expand3x3 = Conv2D(filters=expand3x3_filters, kernel_size=(3, 3), activation='relu', dilation_rate=dilation_rate, padding='same')(squeeze)

    return Concatenate()([expand1x1, expand3x3])

def create_squeezenet_dilated_residual(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='valid')(input_tensor)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # First Residual Block
    residual = fire_module(x, 16, 64, 64, dilation_rate=2)
    residual = Conv2D(64, (1, 1), activation='relu')(residual)
    x = fire_module(x, 16, 64, 64, dilation_rate=2)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Add()([x, residual])

    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Second Residual Block
    residual = fire_module(x, 32, 128, 128, dilation_rate=3)
    residual = Conv2D(128, (1, 1), activation='relu')(residual)
    x = fire_module(x, 32, 128, 128, dilation_rate=3)
    x = Conv2D(128, (1, 1), activation='relu')(x)
    x = Add()([x, residual])

    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Third Residual Block
    residual = fire_module(x, 48, 192, 192, dilation_rate=4)
    residual = Conv2D(192, (1, 1), activation='relu')(residual)
    x = fire_module(x, 48, 192, 192, dilation_rate=4)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Add()([x, residual])

    x = GlobalAveragePooling2D()(x)

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

# Rest of the code remains unchanged


In [3]:
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
input_shape=(244,244,3)
# Assuming 'create_squeezenet_dilated_residual' function returns the model architecture
model = create_squeezenet_dilated_residual((input_shape), num_classes=2)  # Ensure num_classes is set to 2

# Define the optimizer and loss function
optimizer = Adam()
loss_function = CategoricalCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Data Preprocessing and Data Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    "/content/splitted_data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Data Preprocessing for validation data
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0
)

val_generator = val_datagen.flow_from_directory(
    "/content/splitted_data/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Training the model with validation data
epochs = 50
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

try:
    history = model.fit(
        train_generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_generator,
        validation_steps=validation_steps
    )
except Exception as e:
    print("An error occurred during training:", str(e))

# Plot the training and validation losses
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.show()


FileNotFoundError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.profiler import ProfilerSession
from tensorflow.python.framework.ops import get_name_scope

# Create and compile your TensorFlow model
#model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224, 224, 3))
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create a sample input tensor for profiling
input_tensor = tf.random.uniform((1, 224, 224, 3))

# Run a session with profiler
with ProfilerSession() as prof:
    # Run a forward pass with the model
    model(input_tensor)

# Extract statistics from the profiler
stats = prof.profile_ops
total_flops = sum([op.stats.total_float_ops for op in stats.values()])

print(f"Total FLOPS: {total_flops}")

In [ ]:
import matplotlib.pyplot as plt
# Plot the training and validation losses
plt.plot(history.history['loss'],label = 'Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
#plt.title('loss')
plt.legend()
plt.show()

In [ ]:
# Both Validation and Training accuracy is shown here

plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
#plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
# CHECKING THE CONFUSION MATRIX

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
Y_pred = model.predict(test_set)
y_pred = np.argmax(Y_pred ,axis =1)


In [ ]:
print('Classification Report')
target_names = ['aca','n']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

In [ ]:
loss, acc = model.evaluate_generator(test_set, len(test_set))

print ("\n\n================================\n\n")
print ("Loss: {}".format(loss))
print ("Accuracy: {0:.2f} %".format(acc * 100))
print ("\n\n================================\n\n")

test_set.reset()

In [ ]:
result = model.evaluate(test_set,batch_size=128)
print("test_loss, test accuracy",result)

In [ ]:
preds = model.predict(test_set,verbose=1)
predictions = preds.copy()
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1